In [ ]:
import mne
import numpy as np
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, cross_val_score  
from scipy.io import loadmat
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Training 

In [6]:
file  ="C:\\Users\\Luchitha\\Documents\\BCI\\BCI_IV\\dataset\\A01T.gdf"  
mat_file   = "C:\\Users\\Luchitha\\Documents\\BCI\\BCI_IV\\true_labels\\A01T.mat"

In [16]:

# Load EEG
raw_train = mne.io.read_raw_gdf(file, preload=True)
raw_train.pick_types(eeg=True)  # exclude EOG
raw_train.filter(8., 30., fir_design='firwin')  # MI band

# Load true labels for training
y_train = loadmat(mat_file)['classlabel'].flatten() - 1  # 0-3

# Extract events
events_train, event_id = mne.events_from_annotations(raw_train)

# 1. Find all trial start events (768)
trial_starts = np.where(events_train[:, 2] == event_id['768'])[0]

# 2. Build events array for MNE
# MNE expects (n_events, 3): [sample, 0, label], we temporarily set label=0
events_train_manual = np.array([[events_train[i, 0], 0, 0] for i in trial_starts])

# 3. Epoch
epochs_train = mne.Epochs(
    raw_train,
    events_train_manual,
    tmin=2.0,
    tmax=6.0,
    baseline=None,
    preload=True
)

X_train = epochs_train.get_data()  # EEG data


Extracting GDF parameters from C:\Users\Luchitha\Documents\BCI\BCI_IV\dataset\A01T.gdf...
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


C:\Users\Luchitha\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

Used Annotations descriptions: [np.str_('1023'), np.str_('1072'), np.str_('276'), np.str_('277'), np.str_('32766'), np.str_('768'), np.str_('769'), np.str_('770'), np.str_('771'), np.str_('772')]
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 288 events and 1001 origin

In [17]:
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
lda = LinearDiscriminantAnalysis()

pipeline = Pipeline([('CSP', csp), ('LDA', lda)])
pipeline.fit(X_train, y_train)

Computing rank from data with rank=None


    Using tolerance 6.7e-05 (2.2e-16 eps * 25 dim * 1.2e+10  max singular value)
    Estimated rank (data): 25
    data: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('CSP', ...), ('LDA', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,n_components,4
,reg,None
,log,True
,cov_est,'concat'
,transform_into,'average_power'
,norm_trace,False
,cov_method_params,None


# Testing

In [13]:
test = "C:\\Users\\Luchitha\\Documents\\BCI\\BCI_IV\\dataset\\A01E.gdf"  
test_mat = "C:\\Users\\Luchitha\\Documents\\BCI\\BCI_IV\\true_labels\\A01E.mat"

In [18]:



raw_eval = mne.io.read_raw_gdf(test, preload=True)
raw_eval.pick_types(eeg=True)  # only EEG channels
raw_eval.filter(8., 30., fir_design='firwin')  # typical MI band


y_eval = loadmat(test_mat)['classlabel'].flatten() - 1  # map 1-4 -> 0-3


events_eval, event_id = mne.events_from_annotations(raw_eval)


trial_start_idx = np.where(events_eval[:, 2] == event_id['768'])[0]


events_eval_manual = np.array([[events_eval[i, 0], 0, 0] for i in trial_start_idx])


epochs_eval = mne.Epochs(
    raw_eval,
    events_eval_manual,
    tmin=2.0,   # 2s after trial start → cue MI window
    tmax=6.0,   # end of MI
    baseline=None,
    preload=True
)


X_eval = epochs_eval.get_data()


print(X_eval.shape, y_eval.shape)


Extracting GDF parameters from C:\Users\Luchitha\Documents\BCI\BCI_IV\dataset\A01E.gdf...
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 686999  =      0.000 ...  2747.996 secs...


C:\Users\Luchitha\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

Used Annotations descriptions: [np.str_('1023'), np.str_('1072'), np.str_('276'), np.str_('277'), np.str_('32766'), np.str_('768'), np.str_('783')]
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 288 events and 1001 original time points ...
0 bad epochs dropped
(288, 25

In [ ]:
y_pred = pipeline.predict(X_eval)



acc = accuracy_score(y_eval, y_pred)
cm = confusion_matrix(y_eval, y_pred)
report = classification_report(y_eval, y_pred, target_names=['Left', 'Right', 'Feet', 'Tongue'])

print("Evaluation Accuracy:", acc)
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", report)
